In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib
import sqlite3
from sqlite3 import Error

In [6]:
conn = sqlite3.connect('wine_data.sqlite')
c = conn.cursor()


In [12]:
df = pd.read_sql('Select * from wine_data', conn)
df.head(1)

,index,country,description,rating,price,province,title,variety,winery,color
0,0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red


In [15]:
features_df = pd.get_dummies(df, columns = ['country', 'color','variety'])
del features_df['price']
del features_df['index']
del features_df['description']
del features_df['title']
del features_df['province']
del features_df['winery']
features_df.head(1)

,rating,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Brazil,country_Bulgaria,country_Canada,country_Chile,country_Croatia,...,variety_Shiraz,variety_Sparkling Blend,variety_Syrah,variety_Tempranillo,variety_Tempranillo Blend,variety_Torrontés,variety_Verdejo,variety_Viognier,variety_White Blend,variety_Zinfandel
0,87,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#create X and y arrays from the dataset using the .values command

X = features_df.values
y = df['price'].values

In [27]:
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.3, shuffle = False)

In [42]:
model = ensemble.GradientBoostingRegressor(
    n_estimators = 500, #how many decision trees to build
    learning_rate = 0.03, #controls rate at which additional decision trees influes overall prediction
    max_depth = 10, 
    min_samples_split = 12,
    min_samples_leaf = 10, 
    max_features = 0.6,
    loss = 'huber'
)

In [38]:
model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.025, loss='huber', max_depth=10,
             max_features=0.6, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=10, min_samples_split=12,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [30]:
joblib.dump(model, 'xbgWinePrice.pkl')

['xbgWinePrice.pkl']

In [40]:
#Evaluate Results accuracy using Mean Absolute Error

mse = mean_absolute_error(y_train, model.predict(X_train))
print("Training set mean absolute error: %.2f" % mse)

mse = mean_absolute_error(y_test, model.predict(X_test))
print("Test set mean absolute error %2.f" %mse)

Training set mean absolute error: 12.49
Test set mean absolute error 13


In [23]:
model.predict(X_test)

array([22.13378673, 52.93735974, 45.85891441, ..., 30.98555723,
       49.90080063, 63.59716505])

In [41]:
model.score(X_test, y_test)

0.3774487646013486

AttributeError: 'GradientBoostingRegressor' object has no attribute 'evaluate'